In [1]:
%pip install semantic-link-labs --q

StatementMeta(, c4adb53f-770b-4622-8062-37428bf8a7e7, 7, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
import sempy_labs as labs
import sempy_labs.admin as labs_admin
from pyspark.sql import Row
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed


StatementMeta(, c4adb53f-770b-4622-8062-37428bf8a7e7, 9, Finished, Available, Finished)

In [3]:
def log_to_table(level, workspaceid, message):
    now = datetime.datetime.utcnow().isoformat()
    log_df = spark.createDataFrame([Row(timestamp=now, level=level, workspaceId=workspaceid, message=message)])
    log_df.write.format("delta").mode("append").saveAsTable("Logs_RestoreDatasets")

StatementMeta(, c4adb53f-770b-4622-8062-37428bf8a7e7, 10, Finished, Available, Finished)

In [4]:
def backup_model(ds_name, workspace_id):
    try:
        labs.restore_semantic_model(
            dataset=ds_name,
            file_path=f"{ds_name}.abf",
           allow_overwrite=True,
            force_restore=True,
            ignore_incompatibilities=True,
            workspace=workspace_id
        )
        msg = f"Informational: Restore successful for: {ds_name} in workspace {workspace_id}."
        log_to_table("INFO",workspace_id,msg)
        return msg
    except Exception as e:
       msg = f"Restore failed for: {ds_name} in workspace {workspace_id} with error: {e}"
       log_to_table("ERROR",workspace_id,msg)
       return msg

StatementMeta(, c4adb53f-770b-4622-8062-37428bf8a7e7, 11, Finished, Available, Finished)

In [ ]:
workspaces_df = spark.sql(f"Select DISTINCT Workspace_Id,Dataset_Name from models_discovery_status where Status = 'Success'")
try:
      msg = f" Restore Operation Started."
      log_to_table("INFO","N/A",msg)

      workspace_list = [(row['Dataset_Name'], row['Workspace_Id']) for row in workspaces_df.collect()]
      
      #print(f"Datasets to process: {workspace_list}")
      print(f"Informational:  Restore Started.")

      with ThreadPoolExecutor(max_workers=10) as executor:
         futures = [executor.submit(backup_model, ds,ws) for ds,ws in workspace_list]
         for future in as_completed(futures):
            try: 
               print(future.result())  # This will raise any exceptions caught during execution
            except Exception as e:
               print(f"Exception in future: {e}")

      msg = f"Restore Operation Completed."
      log_to_table("INFO","N/A",msg)
      print(f"Informational: Restore Completed.")
except Exception as e:
      print(f"Informational: Restore Failed")
      msg = f"🔴Error in Restorep operation: {e}"
      log_to_table("ERROR","N/A",msg)                